In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import tifffile
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras import layers, models,optimizers,metrics
from tensorflow.keras.optimizers import Adam
import keras.preprocessing.image 
from skimage.transform import resize
from sklearn.preprocessing import *

In [ ]:
TOP_FOLDER1 = "/kaggle/input/aptel-2021-multtemporal/sia-apt_lz/tops1/"
TOP_FOLDER2 = "/kaggle/input/aptel-2021-multtemporal/sia-apt_lz/tops2/"
MASK_FOLDER = "/kaggle/input/aptel-2021-multtemporal/sia-apt_lz/masks/"
TOPSl1 = sorted(TOP_FOLDER1+i for i in os.listdir(TOP_FOLDER1) )
TOPSl2 = sorted(TOP_FOLDER2+i for i in os.listdir(TOP_FOLDER2) )
MASKSl = sorted(MASK_FOLDER+i for i in os.listdir(MASK_FOLDER) )
TOP_FOLDER1 = "/kaggle/input/aptel-2021-multtemporal/sia-apt_uz/tops1/"
TOP_FOLDER2 = "/kaggle/input/aptel-2021-multtemporal/sia-apt_uz/tops2/"
MASK_FOLDER = "/kaggle/input/aptel-2021-multtemporal/sia-apt_uz/masks/"
TOPSu1 = sorted(TOP_FOLDER1+i for i in os.listdir(TOP_FOLDER1) )
TOPSu2 = sorted(TOP_FOLDER2+i for i in os.listdir(TOP_FOLDER2) )
MASKSu = sorted(MASK_FOLDER+i for i in os.listdir(MASK_FOLDER) )

In [ ]:
TOPS1 = TOPSl1[:-5] +TOPSu1[:-5]
TOPS2 = TOPSl2[:-5] +TOPSu2[:-5]
MASKS  = MASKSl[:-5] + MASKSu[:-5]

In [ ]:
VTOPS1 = TOPSl1[-5:] +TOPSu1[-5:]
VTOPS2 = TOPSl2[-5:] +TOPSu2[-5:]
VMASKS  = MASKSl[-5:] + MASKSu[-5:]

In [ ]:
print(len(TOPS1))
print(len(TOPS2))
print(len(MASKS))
print(TOPS1[:5])
print(TOPS2[:5])
print(MASKS[:5])

In [ ]:
TOP_FOLDER1 = "/kaggle/input/d/shivammukhedkar/ker-2122-scaled/tops1/"
TOP_FOLDER2 = "/kaggle/input/d/shivammukhedkar/ker-2122-scaled/tops2/"
MASK_FOLDER = "/kaggle/input/d/shivammukhedkar/ker-2122-scaled/masks/"
TOPSk1 = sorted(TOP_FOLDER1+i for i in os.listdir(TOP_FOLDER1) )
TOPSk2 = sorted(TOP_FOLDER2+i for i in os.listdir(TOP_FOLDER2) )
MASKSk = sorted(MASK_FOLDER+i for i in os.listdir(MASK_FOLDER) )

In [ ]:
VTOPSk1 = TOPSk1[-5:]
VTOPSk2 = TOPSk2[-5:]
VMASKSk = MASKSk[-5:]
TOPSk1 = TOPSk1[:-5]
TOPSk2 = TOPSk2[:-5]
MASKSk = MASKSk[:-5]

In [ ]:
print(len(TOPSk1))
print(len(TOPSk2))
print(len(MASKSk))
print(TOPSk1[:5])
print(TOPSk2[:5])
print(MASKSk[:5])

In [ ]:
VTOP1 = VTOPS1 + VTOPSk1 
VTOP2 = VTOPS2 + VTOPSk2
VMASK = VMASKS + VMASKSk

In [ ]:
print(len(VTOP1))
print(len(VTOP2))
print(len(VMASK))

In [ ]:
tt = 2
imgt= tifffile.imread(TOPS1[tt])
print(imgt.shape)
plt.subplot(2, 2, 1)
plt.imshow(imgt[:,:,2])

imgm= tifffile.imread(MASKS[tt])
print(imgm.shape)
plt.subplot(2, 2, 2)
plt.imshow(imgm)

imgt= tifffile.imread(TOPSk1[tt])
print(imgt.shape)
plt.subplot(2, 2, 3)
plt.imshow(imgt[:,:,2])

imgm= tifffile.imread(MASKSk[tt])
print(imgm.shape)
plt.subplot(2, 2, 4)
plt.imshow(imgm)

In [ ]:
def enco(imgm):
    t = list(imgm.shape)+[1]
    final = np.equal(imgm,1)
    final = final.reshape(t)
    for i in range(2,19):
        cmap = np.equal(imgm,i)
        final = np.concatenate((final,cmap.reshape(t)),axis=2)
    return final

In [ ]:
def deco(imgm):
    img = np.argmax(imgm,axis=-1) +1 
    return img

In [ ]:
img_height = 256
img_width = 256
img_channels=3
mask_height = 256
mask_width = 256
x_train = []
x2_train = []
y_train = []
for i in range(len(TOPS1)):
    img= tifffile.imread(TOPS1[i])
    x_train.append(img)
    img= tifffile.imread(TOPS2[i])
    x2_train.append(img)
    imgm= tifffile.imread(MASKS[i])
    imgm2 = enco(imgm)
    y_train.append(imgm2)
    print(i+1,end="\r")
print(i+1,"done")
x_train = np.array(x_train)
x2_train = np.array(x2_train)
y_train = np.array(y_train,dtype=np.float32)

In [ ]:
img_height = 256
img_width = 256
img_channels=3
mask_height = 256
mask_width = 256
x_train2 = []
x2_train2 = []
y_train2 = []
for i in range(len(TOPSk1)):
    img= tifffile.imread(TOPSk1[i])
    x_train2.append(img)
    img= tifffile.imread(TOPSk2[i])
    x2_train2.append(img)
    imgm= tifffile.imread(MASKSk[i])
    imgm2 = enco(imgm)
    y_train2.append(imgm2)
    print(i+1,end="\r")
print(i+1,"done")
x_train2 = np.array(x_train2)
x2_train2 = np.array(x2_train2)
y_train2 = np.array(y_train2,dtype=np.float32)

In [ ]:
print(x_train.shape)
print(x2_train.shape)
print(y_train.shape)
print(x_train2.shape)
print(x2_train2.shape)
print(y_train2.shape)

In [ ]:
X_train1 = np.concatenate((x_train,x_train2),axis = 0)
X_train2 = np.concatenate((x2_train,x2_train2),axis = 0)
Y_train1 = np.concatenate((y_train,y_train2),axis = 0)
Y_train2 = np.concatenate((np.ones((x_train.shape[0])),np.zeros((x_train2.shape[0]))),axis=0)

In [ ]:
print(X_train1.shape)
print(X_train2.shape)
print(Y_train1.shape)
print(Y_train2.shape)

In [ ]:
img_height = 256
img_width = 256
img_channels=3
mask_height = 256
mask_width = 256
x_test = []
x2_test = []
y_test = []
for i in range(len(VTOP1)):
    img= tifffile.imread(VTOP1[i])
    x_test.append(img)
    img= tifffile.imread(VTOP2[i])
    x2_test.append(img)
    imgm= tifffile.imread(VMASK[i])
    imgm2 = enco(imgm)
    y_test.append(imgm2)
    print(i+1,end="\r")
print(i+1,"done")
x_test = np.array(x_test)
x2_test = np.array(x2_test)
y_test = np.array(y_test,dtype=np.float32)

In [ ]:
y2_test = np.concatenate((np.ones(10),np.zeros(5)),axis=0)

In [ ]:
print(x_test.shape)
print(x2_test.shape)
print(y_test.shape)
print(y2_test.shape)

In [ ]:
# import random
# temp = x_train.shape[0]
# temp2 = x_train2.shape[0]
# li = random.sample(range(0,temp),1)

In [ ]:
# for i in range(len(li)):
#     t = X_train[temp + i]
#     X_train[temp+i] = X_train[li[i]]
#     X_train[li[i]] = t
    
#     t = Y_train2[temp + i]
#     Y_train2[temp+i] = Y_train2[li[i]]
#     Y_train2[li[i]] = t    
    
#     t = Y_train1[temp + i]
#     Y_train1[temp+i] = Y_train1[li[i]]
#     Y_train1[li[i]] = t    

In [ ]:
def jaccard_coef(y_true, y_pred):
    y_true_f = layers.Flatten()(y_true)
    y_pred_f = layers.Flatten()(y_pred)
    intersection = layers.Dot(axes=(1,1))([y_true_f , y_pred_f])
    return (intersection + 1.0) / (layers.Dot(axes=(1,1))([y_true_f , y_true_f]) + layers.Dot(axes=(1,1))([y_pred_f , y_pred_f]) - intersection + 1.0)


def jaccard_coef_loss(y_true, y_pred):
    return 1-jaccard_coef(y_true, y_pred)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice_coeff = (2 * intersection + K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())
    return dice_coeff

def dice_coef_loss(y_true, y_pred):
    return (1 - dice_coef(y_true, y_pred) )
gamma=1
alpha=0.25
def focal_loss_with_logits(logits, targets, y_pred, alpha= 0.25, gamma=1):
    weight_a = alpha * (1 - y_pred) ** gamma * targets
    weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
    return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b

def focal_loss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(),1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))
    loss = focal_loss_with_logits(logits=logits, targets=y_true,alpha=alpha, gamma=gamma, y_pred=y_pred)
    return tf.reduce_mean(loss)

In [ ]:
def euclidean_distance(vect):
    x, y = vect
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def contrastive_loss1(y_true, y_pred):
    margin = 1.0
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean((1 - y_true) * square_pred + y_true * margin_square)


def pair_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred > 0.5, y_true.dtype)))


# tf.keras.metrics.OneHotMeanIoU(num_classes=6)

In [ ]:
def unet_enc(inp1,inp2):
    
    #contraction path
    c11 = layers.Conv2D(32, 3, activation='relu', padding='same')
    c12 = layers.Conv2D(32, 3, activation='relu', padding='same')
    p1 = layers.MaxPooling2D((2, 2))
    
    c21 = layers.Conv2D(64, 3, activation='relu', padding='same')
    c22 = layers.Conv2D(64, 3, activation='relu', padding='same')
    p2 = layers.MaxPooling2D((2, 2))
    
    c31 = layers.Conv2D(128, 3, activation='relu', padding='same')
    c32 = layers.Conv2D(128, 3, activation='relu', padding='same')
    p3 = layers.MaxPooling2D((2, 2)) 
    d4 = layers.Dropout(0.2)
    
    c41 = layers.Conv2D(256, 3, activation='relu', padding='same')
    c42 = layers.Conv2D(256, 3, activation='relu', padding='same')
    p4 = layers.MaxPooling2D((2, 2)) 
    
    c51 = layers.Conv2D(512, 3, activation='relu', padding='same')
    c52 = layers.Conv2D(512, 3, activation='relu', padding='same')
    
    x = c11(inp1)
    x = c12(x)
    s1 = x
    x = p1(x)
    x = c21(x)
    x = c22(x)
    s2 = x
    x = p2(x)
    x = c31(x)
    x = c32(x)
    s3 =x
    x = p3(x)
    x = d4(x)
    x = c41(x)
    x = c42(x)
    s4 = x
    x = p4(x)
    x = c51(x)
    out1 = c52(x)
    
    x = c11(inp2)
    x = c12(x)
    x = p1(x)
    x = c21(x)
    x = c22(x)
    x = p2(x)
    x = c31(x)
    x = c32(x)
    x = p3(x)
    x = d4(x)
    x = c41(x)
    x = c42(x)
    x = p4(x)
    x = c51(x)
    out2 = c52(x)
    
    skip_connections = [s1, s2, s3, s4]
    print("out1 - ",out1.shape)
    print("out2 - ",out2.shape)
    print("skip - ",skip_connections)
    return out1,out2,skip_connections

In [ ]:
def unet_dec(inp, skip_connections):
    
    u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name='cvt1')(inp)
    u6 = layers.concatenate([u6, skip_connections[3]]) 
    c6 = layers.Conv2D(256, 3, activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(256, 3, activation='relu', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='cvt2')(c6)
    u7 = layers.concatenate([u7, skip_connections[2]])
    c7 = layers.Conv2D(128, 3, activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(128, 3, activation='relu', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', name='cvt3')(c7)
    u8 = layers.concatenate([u8, skip_connections[1]]) 
    c8 = layers.Conv2D(64, 3, activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(64, 3, activation='relu', padding='same')(c8)
    d8 = layers.Dropout(0.2)(c8)

    u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', name='cvt4')(d8)
    u9 = layers.concatenate([u9, skip_connections[0]], axis=3) 
    c9 = layers.Conv2D(32, 3, activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(32, 3, activation='relu', padding='same')(c9)

    out = layers.Conv2D(18, (1, 1), activation='softmax')(c9)
#     print("debug1")
#     out_model = models.Model(inputs=[inp, skip_connections], outputs=[out]) # would have to add input layers whose shape very large
#     print("debug2")
#     out_model.summary()

    return out


In [ ]:
coeff = K.variable(1.0)
@tf.function
def reverse_gradient(x, coeff):
    return tf.stop_gradient(coeff * x + (1 - coeff) * x)

In [ ]:
def siamdu_model():
    in1 = layers.Input((img_height,img_width,img_channels), name='Input_1')
    in2 = layers.Input((img_height,img_width,img_channels), name='Input_2')
    p = unet_enc(in1,in2)
    
#     p1w = p1.output
    p1_o = p[0]
    skip_conn =p[2:]
#     p2w = p2.output
    p2_o = p[1]
    print("Shapes:")
#     print("p1.output:", [i for i in p1.output])
    print("p1_o:", p1_o.shape)
    print("s_c:", [i for i in skip_conn[0]])
#     print("p2.output:", [i for i in p2.output])
#     print("p2_o:", [i for i in p2_o])
    out =  unet_dec(p1_o,skip_conn[0])
#     o_o = out.output
    o_o  = out 
    
    p1p = layers.MaxPooling2D((2, 2)) (p1_o)
    p2p = layers.MaxPooling2D((2, 2)) (p2_o)
    flat1 = layers.Flatten()(p1p)
    flat2 = layers.Flatten()(p2p)
    d1 = layers.Dense(256, activation='sigmoid')(flat1)
    d2 = layers.Dense(256, activation='sigmoid')(flat2)
    g1_o = layers.Lambda(reverse_gradient, arguments={'coeff': coeff})(d1)
    g2_o = layers.Lambda(reverse_gradient, arguments={'coeff': coeff})(d2)
    c1 = layers.Dense(128, activation='sigmoid')(g1_o)
    c2 = layers.Dense(128, activation='sigmoid')(g2_o)
    D = layers.Lambda(euclidean_distance)([c1, c2])
    # out_model = models.Model([in1, in2], [c1, D, c2])
    # embout_model = models.Model(in1, p1)
    out_model = models.Model([in1, in2],[o_o,D])
#     out_model.summary()
    return out_model

In [ ]:
model = siamdu_model()

In [ ]:
# tf.keras.utils.plot_model(model)

In [ ]:
def custom_loss(y_true, y_pred):
    if(len(y_true.shape)==2) :
        return contrastive_loss1(y_true, y_pred)
    else:
        return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [ ]:
# def custom_metric(y_true, y_pred):
# #     print(y_true.shape,"tm")
# #     print(y_pred.shape,"pm")
#     if(len(y_true.shape)==2) :
#         return pair_accuracy(y_true, y_pred)
#     else:
#         return jaccard_coef(y_true, y_pred)

In [ ]:
m = metrics.BinaryAccuracy()
def custom_metric(y_true, y_pred):
    if(len(y_true.shape)==2) :
        m.update_state(y_true, y_pred)
        return m.result()
    else:
        return jaccard_coef(y_true, y_pred)

In [ ]:
# checkpoint = tf.keras.callbacks.ModelCheckpoint('LULC1.hdf5', monitor=[custom_metric], verbose=1, save_best_only=True,mode='max')
# early=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=7,min_delta=0.002,restore_best_weights = True)
# callbs = [early,checkpoint]
# callbs = [checkpoint]
callbs = []

In [ ]:
# model.compile(optimizer=Adam(learning_rate=1e-4),loss=custom_loss, metrics=[jaccard_coef,'accuracy'])
# hist = model.fit([x_train,x2_train],y_train,validation_split=0.2,epochs=150,batch_size=16,callbacks = callbs)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss="binary_crossentropy", metrics=[custom_metric])
hist = model.fit([X_train1,X_train2],[Y_train1,Y_train2], validation_data=([x_test,x2_test], [y_test,y2_test]),epochs=200,batch_size=16,callbacks = callbs)

In [ ]:
# with open('/trainHistoryDict', 'wb') as file_pi:
#     pickle.dump(history.history, file_pi)

np.save('history_JL.npy',hist.history) 
# history=np.load('history_JL.npy',allow_pickle='TRUE').item()

In [ ]:
lab = list(hist.history.keys())
print(lab)

In [ ]:
def plotting(dat):
    lab = list(dat.history.keys())
    fig = plt.figure(figsize=(20,6))
    ax1 = fig.add_subplot(121)
    for i in range(len(lab)):
        if("loss" in lab[i]):
            plt.plot(dat.history[lab[i]], label=lab[i])
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    ax1.legend()
    ax2 = fig.add_subplot(122) 
    for i in range(len(lab)):
        if("accuracy" in lab[i] or "conv2d" in lab[i]):
            plt.plot(dat.history[lab[i]], label=lab[i])
    plt.xlabel('Epoch')
    plt.ylabel('metrics')
    ax2.legend()
    plt.savefig('SULC1.png')

In [ ]:
plotting(hist)

In [ ]:
# # loading saved model 
# model = models.load_model('sLl.hdf5',compile=False)
# model.compile(optimizer=Adam(ilearning_rate=1e-4),loss=contrastive_loss1, metrics=['accuracy',pair_accuracy])
# model.save("mo1.h5")
# tf.keras.models.save_model(model, 'my_model.h5')


In [ ]:

# model = models.load_model('/kaggle/input/dfsdfsad/seg_arialRGBP.hdf5',compile=False)
# model.compile(optimizer=Adam(learning_rate=1e-4),loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
print(len(x_test))
print(len(x2_test))
print(len(y_test))
print(len(y2_test))

In [ ]:
y_pred = model.predict([x_test,x2_test])

In [ ]:
len(y_pred)

In [ ]:
y_pred = y_pred[0]

In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score

In [ ]:
y_pred2 = y_pred.round()
y_pred2_flat = y_pred2.flatten()
y_test_flat = y_test.flatten()

In [ ]:
print(accuracy_score(y_test_flat,y_pred2_flat))
print(precision_score(y_test_flat,y_pred2_flat))
print(recall_score(y_test_flat,y_pred2_flat))
print(f1_score(y_test_flat,y_pred2_flat))

In [ ]:
tem = []
tem2 = []
for i in range(len(y_test)):
    tem2.append(deco(y_test[i]))
    tem.append(deco(y_pred[i]))
tem= np.array(tem)
tem2 = np.array(tem2)

In [ ]:
for i in range(1,19):
    tem[0][0][i] =i
    tem2[0][0][i] =i

In [ ]:
print(accuracy_score(tem.flatten(),tem2.flatten()))

In [ ]:
mat = confusion_matrix(tem.flatten(),tem2.flatten())

cm_display = ConfusionMatrixDisplay(confusion_matrix = mat)
fig, ax = plt.subplots(figsize=(15,15))

cm_display.plot(ax=ax)
# plt.show()
plt.savefig('DSmatec.png')
# print("Precision score = ",precision_score(y_tmask[0].flatten(),y_premask[0].flatten()))
# print("Recall score = ",recall_score(y_tmask[0].flatten(),y_premask[0].flatten()))
# print("F1 score = ",f1_score(y_tmask[0].flatten(),y_premask[0].flatten()))